In [2]:
import pandas as pd
import numpy as np
import re

INPUT_FILE = "information_and_organization_journal_articles.csv"
OUTPUT_FILE = "information_and_organization_MASTER_READY.csv"

MASTER_COLUMNS = [
    "URL",
    "Journal_Title",
    "Standardized_Title",
    "month_year",
    "Abstract",
    "Keywords",
    "Author_Name",
    "Standardized_Author",
    "Author_University",
    "Standardized_University",
    "Author_State",
    "Author_Country",
]

NA_LIKE_VALUES = {
    "", " ", "NA", "N/A", "na", "n/a",
    "NULL", "null", "None", "none", "-"
}

def generate_standardized_title(title):
    if pd.isna(title):
        return np.nan
    text = str(title).strip().lower()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s]", "", text)
    return text

def clean_author_level_to_master(input_csv: str, output_csv: str):
    data = pd.read_csv(input_csv, header=None)

    if data.shape[1] != 10:
        raise ValueError(
            f"{input_csv} has {data.shape[1]} columns; expected 10 for the journal_articles file."
        )

    data.columns = [
        "URL",
        "Journal_Title",
        "Article_Title",
        "Volume_Issue",
        "month_year",
        "Abstract",
        "Keywords",
        "Author_name",
        "Author_email",
        "Author_Address",
    ]

    data = data.replace(list(NA_LIKE_VALUES), np.nan)

    for col in data.columns:
        data[col] = data[col].astype("string").str.strip()

    data = data.dropna(subset=["URL", "Article_Title"])

    data = data.drop_duplicates(
        subset=["URL", "Author_name", "Author_email", "Author_Address"],
        keep="first"
    )

    cleaned = pd.DataFrame({
        "URL": data["URL"],
        "Journal_Title": data["Journal_Title"],
        "Standardized_Title": data["Article_Title"].apply(generate_standardized_title),
        "month_year": data["month_year"],
        "Abstract": data["Abstract"],
        "Keywords": data["Keywords"],
        "Author_Name": data["Author_name"],
        "Standardized_Author": np.nan,
        "Author_University": np.nan,
        "Standardized_University": np.nan,
        "Author_State": np.nan,
        "Author_Country": np.nan,
    })[MASTER_COLUMNS]

    cleaned.to_csv(output_csv, index=False)
    print(f"Saved: {output_csv}")
    print(f"Rows: {len(cleaned)}")
    print(f"Unique URLs: {cleaned['URL'].nunique()}")

clean_author_level_to_master(INPUT_FILE, OUTPUT_FILE)

Saved: information_and_organization_MASTER_READY.csv
Rows: 778
Unique URLs: 350


In [5]:
import os
import openai
import pandas as pd
import json
import time

openai.api_key = os.getenv("OPENAI_API_KEY")

INPUT_FILE = "information_and_organization_MASTER_READY.csv"
OUTPUT_FILE = "information_and_organization_ENRICHED.csv"

def extract_author_affiliation(text):
    if not isinstance(text, str) or text.strip() == "":
        return {
            "Author_University": None,
            "Author_State": None,
            "Author_Country": None
        }

    prompt = f"""
Extract the following details from the author affiliation text.

Return ONLY JSON with these exact keys:
Author_University, Author_State, Author_Country

Use official English names.
Standardize country names (US/USA -> United States).
Standardize state names (CA -> California).
If unknown, return null.

Text:
{text}

JSON:
{{
  "Author_University": "Wichita State University",
  "Author_State": "Kansas",
  "Author_Country": "United States"
}}
"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You extract structured academic affiliation data."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        content = response.choices[0].message.content.strip()
        data = json.loads(content)

        return {
            "Author_University": data.get("Author_University"),
            "Author_State": data.get("Author_State"),
            "Author_Country": data.get("Author_Country")
        }

    except Exception:
        return {
            "Author_University": None,
            "Author_State": None,
            "Author_Country": None
        }

df = pd.read_csv(INPUT_FILE)

universities = []
states = []
countries = []

for _, row in df.iterrows():
    combined_text = f"{row.get('Author_Name','')} {row.get('Author_Address','')}"
    result = extract_author_affiliation(combined_text)

    universities.append(result["Author_University"])
    states.append(result["Author_State"])
    countries.append(result["Author_Country"])

    time.sleep(0.4)

df["Author_University"] = universities
df["Author_State"] = states
df["Author_Country"] = countries

df.to_csv(OUTPUT_FILE, index=False)

print("Saved:", OUTPUT_FILE)
print("Rows:", len(df))

Saved: information_and_organization_ENRICHED.csv
Rows: 778


In [6]:
import pandas as pd
import re

INPUT_FILE = "information_and_organization_ENRICHED.csv"
OUTPUT_FILE = "information_and_organization_FOR_STANDARDIZATION.csv"

def normalize_text(text):
    if not isinstance(text, str):
        return None
    text = text.lower().strip()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

df = pd.read_csv(INPUT_FILE)

df["University_norm"] = df["Author_University"].apply(normalize_text)
df["Author_norm"] = df["Author_Name"].apply(normalize_text)

df.to_csv(OUTPUT_FILE, index=False)

print("Saved:", OUTPUT_FILE)
print("Unique universities:", df["University_norm"].nunique())
print("Unique authors:", df["Author_norm"].nunique())

Saved: information_and_organization_FOR_STANDARDIZATION.csv
Unique universities: 0
Unique authors: 576


In [8]:
import pandas as pd

df = pd.read_csv("information_and_organization_journal_articles.csv")

univ_map = (
    df[["University_norm", "Author_University"]]
    .dropna()
    .drop_duplicates()
    .sort_values("University_norm")
)

univ_map["Standardized_University"] = univ_map["Author_University"]

univ_map.to_csv("university_mapping.csv", index=False)

print("Saved: university_mapping.csv")
print("Rows:", len(univ_map))

KeyError: "None of [Index(['University_norm', 'Author_University'], dtype='object')] are in the [columns]"